## Simple illustration of using an MCP tool

In [6]:
# This is needed to run asyncio within a notebook that in itself is already running asyncio
import nest_asyncio
nest_asyncio.apply()

In [7]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent  # New import: from langchain.agents (v1.0+)
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage  # For explicit message formatting

async def main():
    # Fetch tools from MCP server (unchanged)
    client = MultiServerMCPClient({
        "math": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
        }
    })
    tools = await client.get_tools()

    # Bind tools to the model for ReAct reasoning
    model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

    # Create the ReAct agent (replacement for create_react_agent)
    # Uses a default ReAct prompt under the hood; customize via 'prompt' kwarg if needed
    agent = create_agent(model, tools)

    # Invoke with messages (similar to before)
    response = await agent.ainvoke({
        "messages": [HumanMessage(content="What is 15 multiplied by 7, then add 10?")]
    })
    print(response["messages"][-1].content)  # Outputs: ~115 (with step-by-step reasoning)

asyncio.run(main())

15 multiplied by 7 is 105. When you add 10 to this result, you get 115.
